# Tech Intro

## Main tables

In [559]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [560]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = lead_list[18] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [561]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1120 00:31:14.351390 10390 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:44708
I1120 00:31:14.352130 22925 sched.cpp:241] Version: 1.11.0
I1120 00:31:14.352370 10586 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1120 00:31:14.352473 10586 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1120 00:31:14.354116 10587 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-75603


## Read the spec-file from Hadoop and show some data sample

In [562]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+-------+----------------+--------------------+-----+-------+-----------+----------------+
|id     |leads_contact_id|leads_meta_fields_id|code |name   |value      |phone_normalized|
+-------+----------------+--------------------+-----+-------+-----------+----------------+
|1000006|43136815        |null                |PHONE|Телефон|79184399790|79184399790     |
|1000020|43137192        |null                |PHONE|Телефон|79057178996|79057178996     |
|1000022|43136803        |null                |PHONE|Телефон|79084613205|79084613205     |
|1000076|43138201        |null                |PHONE|Телефон|79772967399|79772967399     |
|10001  |34435283        |null                |PHONE|Телефон|79191456206|79191456206     |
|1000186|43138111        |null                |PHONE|Телефон|79371642279|79371642279     |
|1000203|43137998        |null                |PHONE|Телефон|79174403889|79174403889     |
|1000211|43137000        |null                |PHONE|Телефон|79775563193|79775563193     |

## Show columns' properties

In [563]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- leads_contact_id: string (nullable = true)
 |-- leads_meta_fields_id: string (nullable = true)
 |-- code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- value: string (nullable = true)
 |-- phone_normalized: string (nullable = true)



## Total rows (count)

In [564]:
eda_df.count()

635283

## Simple func to execute and show the result of the SQL-instruction

In [565]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [566]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [567]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|635283  |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [568]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,leads_contact_id,leads_meta_fields_id,code,name,value,phone_normalized
0,0,0,635059,14915,0,170,21068


In [569]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,leads_contact_id,leads_meta_fields_id,code,name,value,phone_normalized
0,0,0,0,0,0,28,0


In [570]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,leads_contact_id,leads_meta_fields_id,code,name,value,phone_normalized
0,0,0,0,0,0,480,0


In [571]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
leads_contact_id,0,0,0
leads_meta_fields_id,635059,0,0
code,14915,0,0
name,0,0,0
value,170,28,480
phone_normalized,21068,0,0


In [572]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [573]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
leads_contact_id,0.000,0.000,0.000
leads_meta_fields_id,99.965,0.000,0.000
code,2.348,0.000,0.000
name,0.000,0.000,0.000
value,0.027,0.004,0.076
phone_normalized,3.316,0.000,0.000


### SQL-instructions

In [574]:
#Find dupes or misunderstanding or useless static values 635 283 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+------------------+
|count(DISTINCT id)|
+------------------+
|635283            |
+------------------+

+--------------------------------+
|count(DISTINCT leads_contact_id)|
+--------------------------------+
|617689                          |
+--------------------------------+

+------------------------------------+
|count(DISTINCT leads_meta_fields_id)|
+------------------------------------+
|2                                   |
+------------------------------------+

+--------------------+
|count(DISTINCT code)|
+--------------------+
|4                   |
+--------------------+

+--------------------+
|count(DISTINCT name)|
+--------------------+
|24                  |
+--------------------+

+---------------------+
|count(DISTINCT value)|
+---------------------+
|563901               |
+---------------------+

+--------------------------------+
|count(DISTINCT phone_normalized)|
+--------------------------------+
|546431                          |
+----------------------------

In [575]:
def execute_template(column: str ='id') -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "ID incl."
    :param :string select_instruction: column name
    :return: result of the SQL-operation as a table
    """
    print(execute("SELECT DISTINCT(" + column + ")\
                FROM eda_sql\
                WHERE 1 = 1 \
                ORDER BY " + column + " DESC"))
    return execute("SELECT DISTINCT(" + column + ")\
                    FROM eda_sql\
                    WHERE 1 = 1 \
                    ORDER BY " + column + " ASC")

In [576]:
#Find dupes or misunderstanding or useless static values 635 283 total
for col in eda_df.columns:
    execute_template(col)

+------+
|id    |
+------+
|999995|
|999994|
|999992|
|999989|
|999982|
|99998 |
|999979|
|999977|
|999974|
|999960|
|999956|
|999955|
|999950|
|99995 |
|999949|
|999947|
|999943|
|999941|
|999940|
|99994 |
+------+
only showing top 20 rows

None
+-------+
|id     |
+-------+
|100    |
|10000  |
|1000006|
|100001 |
|1000020|
|1000022|
|1000029|
|100003 |
|1000042|
|1000043|
|1000044|
|1000049|
|1000053|
|1000057|
|1000060|
|1000066|
|1000067|
|1000070|
|1000071|
|1000072|
+-------+
only showing top 20 rows

+----------------+
|leads_contact_id|
+----------------+
|9982181         |
|9943669         |
|9904539         |
|9903901         |
|9903667         |
|9896645         |
|9756943         |
|9731949         |
|9572987         |
|9569741         |
|951085          |
|947175          |
|9358031         |
|9356789         |
|9219869         |
|9188321         |
|9187103         |
|9177179         |
|9168943         |
|9102633         |
+----------------+
only showing top 20 rows

None


## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [577]:
spark.stop()

I1120 00:36:48.959309 10390 sched.cpp:2174] Asked to stop the driver
I1120 00:36:48.959415 10567 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-75603
